In [1]:
!which python
!python --version
!nvidia-smi

/Users/nadavt/opt/anaconda3/envs/jaymody-sps-env/bin/python
Python 3.9.10
zsh:1: command not found: nvidia-smi


In [2]:
# !python -m torch.utils.collect_env

In [3]:
pip install -q torch transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load model & Sanity check
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

draft_tokenizer = AutoTokenizer.from_pretrained("gpt2")
target_tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

draft_model = GPT2LMHeadModel.from_pretrained("gpt2")
target_model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

draft_inputs = draft_tokenizer("Hello, my dog is cute", return_tensors="pt")
target_inputs = target_tokenizer("Hello, my dog is cute", return_tensors="pt")
# Assert that `draft_tokenizer` and `target_tokenizer` are equivalent
assert torch.all(draft_inputs.input_ids == target_inputs.input_ids)
assert torch.all(draft_inputs.attention_mask == target_inputs.attention_mask)

draft_outputs = draft_model(**draft_inputs, labels=draft_inputs["input_ids"])
target_outputs = target_model(**target_inputs, labels=target_inputs["input_ids"])


loss = draft_outputs.loss
logits = draft_outputs.logits
logits.shape

/Users/nadavt/opt/anaconda3/envs/jaymody-sps-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 6, 50257])

In [5]:
# Generate
draft_outputs = draft_model.generate(draft_inputs.input_ids, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False)
target_outputs = target_model.generate(target_inputs.input_ids, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False)
print("draft_outputs.sequences == target_outputs.sequences:")
print(draft_outputs.sequences == target_outputs.sequences)

print("draft_outputs.sequences:")
print(draft_tokenizer.batch_decode(draft_outputs.sequences))

print("target_outputs.sequences:")
print(target_tokenizer.batch_decode(target_outputs.sequences))

# print("draft_outputs:")
# print(draft_outputs)
# print("====")
# transition_scores = draft_model.compute_transition_scores(
#     draft_outputs.sequences, draft_outputs.scores, normalize_logits=True
# )
# print("transition_scores:")
# print(transition_scores)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


draft_outputs.sequences == target_outputs.sequences:
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False]])
draft_outputs.sequences:
["Hello, my dog is cute. I'm not sure if she's a puppy or not. I'm not sure if she's"]
target_outputs.sequences:
["Hello, my dog is cute. I'm going to take him to the park today. I'm going to take him to the"]


In [6]:
K = 3
additional_input_ids = torch.arange(K).unsqueeze(0).repeat(3, 1)
print("additional_input_ids.shape:", additional_input_ids.shape)
print("additional_input_ids:")
additional_input_ids

additional_input_ids.shape: torch.Size([3, 3])
additional_input_ids:


tensor([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]])

In [7]:
torch.tile(draft_inputs.input_ids, (K, 1))

tensor([[15496,    11,   616,  3290,   318, 13779],
        [15496,    11,   616,  3290,   318, 13779],
        [15496,    11,   616,  3290,   318, 13779]])

In [8]:
# Verifying: generate(**inputs, ...) == generate(inputs.input_ids, ...)
print("Is attention_mask is unnecessary?", torch.all(draft_model.generate(draft_inputs.input_ids, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False).sequences == draft_model.generate(**draft_inputs, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False).sequences))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Is attention_mask is unnecessary? tensor(True)


In [9]:
from typing import Any
from torch import Tensor
from collections.abc import Iterable


def print_shapes(obj: Any, level: int = 0):
    padding = "--" * level
    if isinstance(obj, Tensor):
        print(f"{padding}{obj.shape}")
    elif isinstance(obj, dict):
        for k, v in obj.items():
            print(f"{padding}{k}")
            print_shapes(v, level + 1)
    elif isinstance(obj, str):
        print(f"{padding}type {type(obj)} is not supported")
    elif isinstance(obj, Iterable):
        for x in obj:
            print_shapes(x, level + 1)
    else:
        print(f"{padding}type {type(obj)} is not supported")


# # Test cases
# print_shapes(["test", Tensor([1, 2, 3]), {"key": Tensor([4, 5, 6])}])
# print("====")
# print_shapes("This is a string")
# print("====")
# print_shapes((1, 2, Tensor([7, 8, 9]), {"nested": [Tensor([10, 11, 12]), "string"]}))


In [10]:
import transformers

assert isinstance(draft_inputs, transformers.tokenization_utils_base.BatchEncoding)

In [11]:
print_shapes(draft_outputs)

sequences
--torch.Size([1, 26])
scores
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])


In [12]:
type(draft_outputs.scores), len(draft_outputs.scores)

(tuple, 20)

In [13]:
print(draft_outputs.scores[-1].shape)
print(draft_outputs.scores[-1].min())
print(draft_outputs.scores[-1].max())

torch.Size([1, 50257])
tensor(-156.0659)
tensor(-112.9710)


In [14]:
draft_tokenizer.batch_decode(draft_outputs.sequences)

["Hello, my dog is cute. I'm not sure if she's a puppy or not. I'm not sure if she's"]

In [15]:
import numpy as np


# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if draft_model.config.is_encoder_decoder else draft_inputs.input_ids.shape[1]
generated_tokens = draft_outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {draft_tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

NameError: name 'transition_scores' is not defined

Batching

In [82]:
import torch

print("Let's see how to create a batch of inputs with different lengths...", end="\n---\n")
print("Here is how the input_ids look like. It is a tensor of shape (batch_size=1, sequence_length):")
print(f"{draft_inputs.input_ids.shape=}")
print(f"{draft_inputs.input_ids=}")

print("---")
print("We want to create something similar... First, here is how we can concatenate new tokens to a sequence:")
# Initial sequence and new token
seq = torch.arange(3, dtype=torch.float).view(1, -1)  # Shape: (batch_size=1, seq_len=3)
new_token = torch.Tensor([3]).view(1, -1)  # Shape: (batch_size=1, seq_len=1)
new_seq = torch.cat((seq, new_token), dim=-1)  # Shape: (batch_size=1, seq_len=4)
print(f"{seq=}")
print(f"{seq.shape=}")
print()
print(f"{new_token=}")
print(f"{new_token.shape=}")
print()
print(f"{new_seq=}")
print(f"{new_seq.shape=}")

print("---")
print("=== NESTED TENSORS ===")
batch: Tensor = torch.nested.nested_tensor([seq.squeeze(), new_seq.squeeze()])
batch_reversed: Tensor = torch.nested.nested_tensor([new_seq.squeeze(), seq.squeeze()])
print(f"{batch=}")
print(f"{batch_reversed=}")

print("---")
print("Unfortunately, converting a nested tensor to a padded tensor is always left-padded:")
padded_batch: Tensor = torch.nested.to_padded_tensor(batch, padding=torch.nan)
padded_batch_reversed: Tensor = torch.nested.to_padded_tensor(batch_reversed, padding=torch.nan)
print(f"{padded_batch=}")
print(f"{padded_batch.shape=}")
print()
print(f"{padded_batch_reversed=}")
print(f"{padded_batch_reversed.shape=}")

print("---")
print("=== LEFT PADDING ===")
print("Padding the original sequence to the same length as new_seq:")
pad_size = new_seq.size(1) - seq.size(1)
print(f"{pad_size=}")
pad_token_id = eos_token_id = torch.nan
padded_seq = torch.nn.functional.pad(seq, (pad_size, 0), value=pad_token_id)  # Left padding
print(f"{padded_seq=}")

print("Creating a batch with padded_seq and new_seq:")
padded_batch = torch.stack([padded_seq.squeeze(), new_seq.squeeze()], dim=0)
print(f"{padded_batch=}")
print(f"{padded_batch.shape=}")

Let's see how to create a batch of inputs with different lengths...
---
Here is how the input_ids look like. It is a tensor of shape (batch_size=1, sequence_length):
draft_inputs.input_ids.shape=torch.Size([1, 6])
draft_inputs.input_ids=tensor([[15496,    11,   616,  3290,   318, 13779]])
---
We want to create something similar... First, here is how we can concatenate new tokens to a sequence:
seq=tensor([[0., 1., 2.]])
seq.shape=torch.Size([1, 3])

new_token=tensor([[3.]])
new_token.shape=torch.Size([1, 1])

new_seq=tensor([[0., 1., 2., 3.]])
new_seq.shape=torch.Size([1, 4])
---
=== NESTED TENSORS ===
batch=nested_tensor([
  tensor([0., 1., 2.]),
  tensor([0., 1., 2., 3.])
])
batch_reversed=nested_tensor([
  tensor([0., 1., 2., 3.]),
  tensor([0., 1., 2.])
])
---
Unfortunately, converting a nested tensor to a padded tensor is always left-padded:
padded_batch=tensor([[0., 1., 2., nan],
        [0., 1., 2., 3.]])
padded_batch.shape=torch.Size([2, 4])

padded_batch_reversed=tensor([[0., 